In [13]:
import pandas as pd
import pylab as pl
import numpy as np
from sklearn import preprocessing
%matplotlib inline 
import matplotlib.pyplot as plt

In [14]:
import pickle
import pandas as pd
with open('alexnet.fea', 'rb') as f:
    alex_features = pickle.load(f)

train_metadata_df = pd.read_csv('dicom_metadata_train_merged.csv') # Took it from kaggle notebooks
import torch

alex_np_fea = [ elem.numpy()[0] for elem in alex_features ]


mult_lab = train_metadata_df.iloc[:, 64:]

In [15]:
alex_features[0].numpy()[0].shape

(1000,)

In [16]:
# multiclass iterative_train_test_split
# http://scikit.ml/api/skmultilearn.model_selection.iterative_stratification.html#module-skmultilearn.model_selection.iterative_stratification
#!pip install scikit-multilearn
from skmultilearn.model_selection import iterative_train_test_split
X_train, y_train, X_test, y_test = iterative_train_test_split(np.array(alex_np_fea), mult_lab.to_numpy(), test_size = 0.2)

In [17]:
# Check if the classes in the train set are distributed the same way as in full data
print("Size of data:", len(mult_lab))
print("Size of train set", len(y_train))
#print(mult_lab['Abdomen'].sum())      # number of abdomens in data
#print(mult_lab.to_numpy()[:,0].sum()) # number of abdomens in data
#print(y_train[:,0].sum())             # number of abdomens in train set

print("Fraction of each bodypart in train set and in full data:")
for i, k in enumerate(mult_lab.keys()):
    print(i, k.ljust(14,' '), round(y_train[:,i].sum()/len(y_train), 2), round(mult_lab[k].sum()/len(mult_lab), 2))

Size of data: 1738
Size of train set 1387
Fraction of each bodypart in train set and in full data:
0 Abdomen        0.05 0.05
1 Ankle          0.03 0.03
2 Cervical Spine 0.05 0.05
3 Chest          0.43 0.42
4 Clavicles      0.01 0.01
5 Elbow          0.01 0.01
6 Feet           0.04 0.04
7 Finger         0.01 0.01
8 Forearm        0.01 0.01
9 Hand           0.04 0.04
10 Hip            0.01 0.01
11 Knee           0.06 0.06
12 Lower Leg      0.02 0.01
13 Lumbar Spine   0.05 0.05
14 Others         0.07 0.07
15 Pelvis         0.04 0.04
16 Shoulder       0.02 0.02
17 Sinus          0.01 0.01
18 Skull          0.01 0.01
19 Thigh          0.01 0.01
20 Thoracic Spine 0.04 0.04
21 Wrist          0.06 0.06


<h2 id="Standardization">Standardization</h2>

In [23]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(X_train)
#scaler.mean_[:3]
#scaler.scale_[:3]
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)
#X_train_scaled[:3]

<h2 id="MultiOutputClassifier">MultiOutputClassifier</h2>

In [20]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
clf = MultiOutputClassifier(LogisticRegression(solver='liblinear')).fit(X_train_scaled, y_train)


In [ ]:
y_pred = clf.predict(X_test)

In [51]:
from sklearn.metrics import accuracy_score
#accuracy_score(y_true, y_pred, *, normalize=True, sample_weight=None)[source]¶
#accuracy_score(y_test, y_pred)
for i, k in enumerate(mult_lab.keys()):
    print(k.ljust(14,' '), accuracy_score(y_test[:,i], y_pred[:,i]))

Abdomen        0.9629629629629629
Ankle          0.9686609686609686
Cervical Spine 0.9914529914529915
Chest          0.42450142450142453
Clavicles      0.9943019943019943
Elbow          0.9857549857549858
Feet           0.9686609686609686
Finger         0.9943019943019943
Forearm        0.9943019943019943
Hand           0.9772079772079773
Hip            0.9857549857549858
Knee           0.9772079772079773
Lower Leg      0.9857549857549858
Lumbar Spine   0.9515669515669516
Others         0.9316239316239316
Pelvis         0.9743589743589743
Shoulder       0.9772079772079773
Sinus          0.9857549857549858
Skull          1.0
Thigh          0.9943019943019943
Thoracic Spine 0.9686609686609686
Wrist          0.9743589743589743


In [24]:
from sklearn.metrics import f1_score
print('F1-scores:')
for i, k in enumerate(mult_lab.keys()):
#for i in range(y_pred.shape[1]):
    print(k.ljust(14,' '), f1_score(y_test[:,i], y_pred[:,i]))


F1-scores:
Abdomen        0.4347826086956522
Ankle          0.0
Cervical Spine 0.896551724137931
Chest          0.5943775100401607
Clavicles      0.0
Elbow          0.0
Feet           0.717948717948718
Finger         0.5
Forearm        0.5
Hand           0.6363636363636364
Hip            0.0
Knee           0.7777777777777778
Lower Leg      0.0
Lumbar Spine   0.0
Others         0.0
Pelvis         0.6666666666666666
Shoulder       0.0
Sinus          0.0
Skull          1.0
Thigh          0.5
Thoracic Spine 0.2666666666666667
Wrist          0.7906976744186046


In [45]:
from sklearn.metrics import confusion_matrix, classification_report
#from sklearn.metrics import roc_auc_score

for i, k in enumerate(mult_lab.keys()):
    print(k)
    print(confusion_matrix(y_test[:,i],y_pred[:,i]))
    #print(classification_report(y_test[:,i],y_pred[:,i]))

print(classification_report(y_test[:,1],y_pred[:,1]))
# many bodyparts have TN (true negative) = 0, recall 0.00 for class 1, therefore f1 score cannot be computed

Abdomen
[[333   0]
 [ 13   5]]
Ankle
[[340   0]
 [ 11   0]]
Cervical Spine
[[335   0]
 [  3  13]]
Chest
[[  1 202]
 [  0 148]]
Clavicles
[[349   0]
 [  2   0]]
Elbow
[[346   0]
 [  5   0]]
Feet
[[326   9]
 [  2  14]]
Finger
[[348   0]
 [  2   1]]
Forearm
[[348   0]
 [  2   1]]
Hand
[[336   0]
 [  8   7]]
Hip
[[346   0]
 [  5   0]]
Knee
[[329   0]
 [  8  14]]
Lower Leg
[[346   0]
 [  5   0]]
Lumbar Spine
[[334   0]
 [ 17   0]]
Others
[[327   0]
 [ 24   0]]
Pelvis
[[333   4]
 [  5   9]]
Shoulder
[[343   0]
 [  8   0]]
Sinus
[[346   0]
 [  5   0]]
Skull
[[349   0]
 [  0   2]]
Thigh
[[348   0]
 [  2   1]]
Thoracic Spine
[[338   0]
 [ 11   2]]
Wrist
[[325   5]
 [  4  17]]
              precision    recall  f1-score   support

           0       0.97      1.00      0.98       340
           1       0.00      0.00      0.00        11

    accuracy                           0.97       351
   macro avg       0.48      0.50      0.49       351
weighted avg       0.94      0.97      0.95       35

/home/anka/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/anka/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/anka/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [48]:
# investigating agregate metrics according to 
# https://towardsdatascience.com/evaluating-multi-label-classifiers-a31be83da6ea
classification_report(
    y_test[:,:2],
    y_pred[:,:2],
    output_dict=False,
    target_names=mult_lab.keys()[:2]
    #target_names=list(mult_lab.keys())[:2]
)

/home/anka/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/anka/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/anka/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


'              precision    recall  f1-score   support\n\n     Abdomen       1.00      0.28      0.43        18\n       Ankle       0.00      0.00      0.00        11\n\n   micro avg       1.00      0.17      0.29        29\n   macro avg       0.50      0.14      0.22        29\nweighted avg       0.62      0.17      0.27        29\n samples avg       0.01      0.01      0.01        29\n'